# TextCNN
1. 预训练词向量
2. TextCNN网络

In [1]:
from google.colab import 
drive.mount('/content/drive')

Mounted at /content/drive


# Content Preprocessing

In [26]:
import bz2
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import jieba

## word vector

从词向量预训练模型中建立词到向量映射

In [15]:
with bz2.open('/content/drive/MyDrive/Colab Notebooks Project/NLP/TouTiao_text/sgns.weibo.word.bz2') as f:
    info, *content = [x.decode('utf-8') for x in f.readlines()]

In [79]:
word_to_vector = {}

for ls in content:
    word, *vector = ls.split()
    word_to_vector[word] = np.array(vector).astype(np.float64)

vector_dim = int(info.split()[1])
word_to_vector['UNK'] = np.random.random(size=vector_dim)
word_to_vector['PAD'] = np.random.random(size=vector_dim)

## training data

对训练数据的词转化成对应的id，如果预训练没有的词则统一变成‘UNK’标识 

In [34]:
df_data = pd.read_csv('/content/drive/MyDrive/Colab Notebooks Project/NLP/TouTiao_text/train.csv')
df_data['sentence_cut'] = df_data['sentence'].apply(jieba.lcut)
df_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 53360 entries, 0 to 53359
Data columns (total 5 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   id            53360 non-null  int64 
 1   label         53360 non-null  int64 
 2   label_desc    53360 non-null  object
 3   sentence      53360 non-null  object
 4   sentence_cut  53360 non-null  object
dtypes: int64(2), object(3)
memory usage: 2.0+ MB


In [81]:
word_to_id = {'UNK': 0, 'PAD': 1}
id_to_word = {0: 'UNK', 1: 'PAD'}
id_for_sentence = []

id = 2
for sentence in df_data['sentence_cut']:
    id_list = []
    for word in sentence:
        
        if word not in word_to_id:
            word_to_id[word] = id
            id_to_word[id] = word
            id += 1
        
        if word in word_to_vector:
            id_list.append(word_to_id[word])
        else:
            id_list.append(word_to_id['UNK'])

    id_for_sentence.append(id_list)

df_data['id_for_sentence'] = id_for_sentence

In [91]:
df_data['id_for_sentence']

0        [2, 3, 4, 0, 6, 7, 8, 9, 10, 4, 11, 12, 7, 13,...
1        [0, 25, 26, 27, 28, 29, 30, 0, 32, 33, 34, 35,...
2        [43, 44, 33, 45, 12, 46, 7, 47, 48, 49, 12, 7,...
3                  [56, 35, 57, 58, 59, 60, 0, 62, 63, 23]
4               [64, 37, 65, 66, 7, 0, 68, 69, 70, 71, 72]
                               ...                        
53355    [464, 508, 331, 0, 20314, 1101, 88, 1101, 1698...
53356        [1577, 5745, 174, 92, 52371, 60429, 37, 3462]
53357    [67812, 956, 67813, 67814, 1444, 75, 14028, 80...
53358    [127, 117, 663, 37, 550, 4663, 7, 12096, 1016,...
53359    [11794, 45079, 43303, 448, 7, 6323, 4932, 333,...
Name: id_for_sentence, Length: 53360, dtype: object

## embedding matrix